## Nilearn
Topics covered:
* Visualization 
* Loading and masking data
* Downloading a test dataset
* some rsfMRI resources
* ICA

In [ ]:
import os
from glob import glob
import pandas
import nibabel as ni
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load some data

wdir = os.path.join(os.getcwd(),'stuff')
mni = os.path.join(wdir,'MNI152_T1_1mm_brain.nii')
mask = os.path.join(wdir,'ADNI_GM_mask_1mm_nocereb.nii.gz')
roi = os.path.join(wdir,'inf_temp.nii.gz')

#### Plotting

One of the most frequent uses I make of nilearn is its visualization capabilities. It's great at quick plotting of neuroimages

In [ ]:
from nilearn import plotting
img = ni.load(mni)

plt.close()
plotting.plot_img(img)
plt.show()

In [ ]:
plt.close()
plotting.plot_roi(ni.load(roi),img)
plt.show()

It's also excellent for quickly loading and masking neuroimaging data and preparing them for analysis

#### Loading and masking

In [ ]:
# here's a bunch of scans
# NOTE: These are on my computer, but try with some images on yours!
jdir = '/Users/jakevogel/Science/tau/tmp/tmp_in/'
scans = glob(os.path.join(jdir,'*.ni*')) 
scans

In [ ]:
# we can load them into a single 4D volume very easily with nilearn
from nilearn import image
image_4d = image.load_img(scans)

In [ ]:
image_4d.shape

In [ ]:
# in order to use the data, we can easily convert it to a 4D numpy array
dat = image_4d.get_data()

In [ ]:
# These images are 1mm3, which means there are 7~ million voxels
len(dat[:,:,:,0].flat)

In [ ]:
# We don't care about most of these. 
# So what if we mask the image do only get voxels we care about
img1 = ni.load(scans[1])
mask_img = ni.load(mask)
plt.close()
plotting.plot_roi(mask_img,img1)
plt.show()

In [ ]:
# nilearn makes this very easy
mask_obj = input_data.NiftiMasker(mask_img) # create a mask object from our mask image
masked_data = mask_obj.fit_transform(image_4d)

In [ ]:
# Now, the data has been masked (only about 700k voxels now), but it has also been 
# transformed into a subject x feature (voxel) array
masked_data.shape

#### loading play datasets

In [ ]:
# Nilearn comes with a fairly large sets of structural and fmri data for you to play and 
# experiment with. Here, I will download a single 4D fmri volume from their adhd dataset
from nilearn import datasets
dataset = datasets.fetch_adhd(n_subjects=1) # download dataset


In [ ]:
# lots of information is available about the datasets
dataset.description

In [ ]:
# for example, here are the confound data for this subject's functional scan
pandas.read_csv(dataset.confounds[0],sep='\t').head()

In [ ]:
# load the image
fmri = image.load_img(dataset.func)
fmri.shape

#### extracting and viewing timeseries

In [ ]:
# define some coordinates in the DMN
dmn_coords = [(0, -52, 18), (-46, -68, 32), (46, -68, 32), (1, 50, -5)]
labels = [
          'Posterior Cingulate Cortex',
          'Left Temporoparietal junction',
          'Right Temporoparietal junction',
          'Medial prefrontal cortex',
         ]

In [ ]:
# draw spheres around the coordinates and use this as a mask
masker = input_data.NiftiSpheresMasker(
    dmn_coords, radius=8,
    detrend=True, standardize=True,
    low_pass=0.1, high_pass=0.01, t_r=2.5,
    memory='nilearn_cache', memory_level=1, verbose=2)

In [ ]:
# extract timeseries from these coordinates, adjusting for confounds
time_series = masker.fit_transform(dataset.func[0],
                                   confounds=dataset.confounds[0])

In [ ]:
# view the time-series
plt.close()
for time_serie, label in zip(time_series.T, labels):
    plt.plot(time_serie, label=label)

plt.title('Default Mode Network Time Series')
plt.xlabel('Frame #')
plt.ylabel('Normalized signal')
plt.legend()
plt.tight_layout()
plt.show()

#### plotting connectomics

In [ ]:
# We can also use nilearn to calculate and plot connectomes
from nilearn.connectome import ConnectivityMeasure
connectivity_measure = ConnectivityMeasure(kind='partial correlation')
partial_correlation_matrix = connectivity_measure.fit_transform(
    [time_series])[0]

In [ ]:
plt.close()
sns.heatmap(partial_correlation_matrix)
plt.show()

In [ ]:
plt.close()
plotting.plot_connectome(partial_correlation_matrix, 
                         dmn_coords,title="Default Mode Network Connectivity")
plt.show()

#### Maybe a little ICA?

In [ ]:
# this could take awhile
from nilearn.decomposition import CanICA
ica_obj = CanICA(n_components=20, random_state=123)
ica = ica_obj.fit(fmri)

In [ ]:
# convert the resulting components to an image format, for easy viewing and interpretation
components_img = ica.masker_.inverse_transform(ica.components_)

# then plot
plt.close()
plotting.plot_prob_atlas(components_img)
plt.show()

In [ ]:
plt.close()
from nilearn.image import iter_img
for i, cur_img in enumerate(iter_img(components_img)):
    plotting.plot_stat_map(cur_img, display_mode='z', title='IC %d'%i, 
                           cut_coords=1, colorbar=False)
plt.show()

Nilearn has fantastic documentation. I encourage you to browse through their examples: http://nilearn.github.io/auto_examples/index.html

If you have any further questions or think I can be of help, my brainhack slack handle is jvogel